# 类元编程

In [1]:
class Dog:
    def __init__(self, name, weight, owner):
        self.name = name
        self.weight = weight
        self.owner = owner

In [2]:
rex = Dog('Rex', 30, 'Bob')

In [3]:
rex

In [4]:
def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split()
    except AttributeError:
        pass
    field_names = tuple(field_names)
    
    def __init__(self, *args, **kwargs):
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)
            
    def __iter__(self):    # 迭代的时候与__slots__有关
        for name in self.__slots__:
            yield getattr(self, name)
            
    def __repr__(self):    # 字符串表示形式也与__slots__有关
        values = ', '.join('{}={!r}'.format(*i) for i in zip(self.__slots__, self))
        # 利用了生成器函数zip，self是可迭代对象，元组拆包，格式规范微语言
        return '{}({})'.format(self.__class__.__name__, values)
    
    cls_attrs = dict(__slots__ = field_names,    # __slots__类属性设为field_names
                     __init__=__init__, 
                     __iter__=__iter__, 
                     __repr__=__repr__)
    
    return type(cls_name, (object,), cls_attrs)

In [5]:
Dog = record_factory('Dog', 'name weight owner')

In [6]:
rex = Dog('Rex', 30, 'Bob')

In [7]:
rex

Dog(name='Rex', weight=30, owner='Bob')

In [8]:
name, weight, _ = rex

In [9]:
name, weight

('Rex', 30)

In [10]:
"{2}'s dog weighs {1}kg'".format(*rex)

"Bob's dog weighs 30kg'"

In [11]:
rex.weight = 32

In [12]:
rex

Dog(name='Rex', weight=32, owner='Bob')

In [13]:
Dog.__mro__

(__main__.Dog, object)

## 定制描述符的类装饰器

In [14]:
import abc


class AutoStorage:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
        
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)
        
    def __delete__(self, instance):
        delattr(instance, self.storage_name)
            
            
class Validated(abc.ABC, AutoStorage):
    
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
        
    @abc.abstractmethod
    def validate(self, instance, value):
        """return validated value or raise ValueError"""
        
        
class Quantity(Validated):
    """a number greater than zero"""
    
    def validate(self, instance, value):
        if value <= 0 :
            raise ValueError('must be > 0')
        else:
            return value
        

class NonBlank(Validated):
    """a string with at least one non-space character"""
    
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value
    
    
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [15]:
LineItem.weight.storage_name

'_Quantity#0'

#### 把被装饰对象的名称绑定给装饰器返回的对象

In [16]:
def entity(cls):    # 类装饰器用于审查和修改刚创建的类，甚至替换成其他类
    for key, attr in cls.__dict__.items():
        if isinstance(attr, Validated):
            type_name = type(attr).__name__
            attr.storage_name = '_{}#{}'.format(type_name, key)
    return cls

In [17]:
@entity
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [18]:
raisin = LineItem('Golden raisin', 10, 6.95)

In [19]:
vars(raisin)

{'_NonBlank#description': 'Golden raisin',
 '_Quantity#weight': 10,
 '_Quantity#price': 6.95}

In [20]:
dir(raisin)[:3]

['_NonBlank#description', '_Quantity#price', '_Quantity#weight']

In [21]:
LineItem.description.storage_name

'_NonBlank#description'

In [22]:
raisin.description

'Golden raisin'

In [23]:
getattr(raisin, '_NonBlank#description')

'Golden raisin'

In [ ]:
# evalsupport.py



print('<[100]> evalsupport module start')


def deco_alpha(cls):
    print('<[200]> deco_alpha')
    
    def inner_1(self):
        print('<[300]> deco_alpha:inner_1')
    
    cls.method_y = inner_1
    return cls


class MetaAleph(type):    # 注意！MetaAleph继承自type元类，继承了构建类的能力
    print('<[400]> MetaAleph body')
    
    def __init__(cls, name, bases, dic):
        print('<[500]> MetaAleph.__init__')
        
        def inner_2(self):
            print('<[600]> MetaAleph.__init__:inner_2')
        
        cls.method_z = inner_2


print('<[700]> evalsupport module end')

In [ ]:
# evaltime.py



from evaltime import deco_alpha

print('<[1]> evaltime module start')


class ClassOne():
    print('<[2]> ClassOne bode')
    
    def __init__(self):
        print('<[3]> ClassOne.__init__')
        
    def __del__(self):
        print('<[4]> ClassOne.__del__')
        
    def method_x(self):
        print('<[5]> ClassOne.method_x')
    
    class ClassTwo(object):
        print('<[6]> ClassTwo body')
        
        
@deco_alpha
class ClassThree():
    print('<[7]> ClassThree body')
    
    def method_y(self):
        print('<[8]> ClassThree.method_y')
        
        
class ClassFour(ClassThree):
    print('<[9]> ClassFour body')
    
    def method_y(self):
        print('<[10]> ClassFour.method_y')
        
        
if __name__ == '__main__':
    print('<[11]> ClassOne tests', 30 * '.')
    one = ClassOne()
    one.method_x()
    print('<[12]> ClassThree tests', 30 * '.')
    three = ClassThree()
    three.method_y()
    print('<[13]> ClassFour tests', 30 * '.')
    four = ClassFour()
    four.method_y()
    
    
print('<[14]> evaltime module end')

## 元类的基础知识

In [24]:
'spam'.__class__

str

In [25]:
str.__class__

type

In [26]:
LineItem.__class__

type

In [27]:
type.__class__

type

In [28]:
from collections import abc

abc.Iterable.__class__

abc.ABCMeta

In [29]:
import abc

abc.ABCMeta.__class__

type

In [30]:
abc.ABCMeta.__mro__

(abc.ABCMeta, type, object)

In [31]:
LineItem.__mro__

(__main__.LineItem, object)

In [ ]:
# evaltime_meta.py 



from evalsupport import deco_alpha
from evalsupport import MetaApleph

print('<[1]> evaltime_meta module start')


@deco_alpha
class ClassThree():
    print('<[2]> ClassThree body')
    
    def method_y(self):
        print('<[3]> ClassThree.method_y')
        
        
class ClassFour(ClassThree):
    print('<[4]> ClassFour body')
    
    def method_y(self):
        print('<[5]> ClassFour.method_y')
        
        
class ClassFive(metaclass=MetaApleph):
    print('<[6]> ClassFive body')
    
    def __init__(self):
        print('<[7]> ClassFive.__init__')
        
    def method_z(self):    # 因为ClassFive是元类MetaApleph的实例，所以构建ClassFive类时会调用元类的__init__方法，ClassFive.method_z被替换成inner_2函数
        print('<[8]> ClassFive.method_z')
        
        
class ClassSix(ClassFive):
    print('<[9]> ClassSix body')
    
    def method_z(self):
        print('<[10]> ClassSix.method_z')
        
        
if __name__ == '__main__':
    print('<[11]> ClassThree tests', 30 * '.')
    three = ClassThree()
    three.method_y()
    print('<[12]> Classfour tests', 30 * '.')
    four = ClassFour()
    four.method_y()
    print('<[13]> ClassFive tests', 30 * '.')
    five = ClassFive()
    five.method_z()
    print('<[14]> ClassSix tests', 30 * '.')
    six = ClassSix()
    six.method_z()
      
print('<[15]> evaltime_meta module end')


class MetaAleph(type):    # 注意！MetaAleph继承自type元类，继承了构建类的能力
    print('<[400]> MetaAleph body')
    
    def __init__(cls, name, bases, dic):
        print('<[500]> MetaAleph.__init__')
        
        def inner_2(self):
            print('<[600]> MetaAleph.__init__:inner_2')
        
        cls.method_z = inner_2

## 定制描述符的元类

In [32]:
import abc


class AutoStorage:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
        
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)
        
    def __delete__(self, instance):
        delattr(instance, self.storage_name)
            
            
class Validated(abc.ABC, AutoStorage):
    
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
        
    @abc.abstractmethod
    def validate(self, instance, value):
        """return validated value or raise ValueError"""
        
        
class Quantity(Validated):
    """a number greater than zero"""
    
    def validate(self, instance, value):
        if value <= 0 :
            raise ValueError('must be > 0')
        else:
            return value
        

class NonBlank(Validated):
    """a string with at least one non-space character"""
    
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value


class EntityMeta(type):
    """元类，用于创建带有验证字段的业务实体"""
    # 这个验证字段是指，在构建类时，验证类的类属性attr是否为Validated类的子类，以此来自动设置这个描述符实例的储存属性名称，
    # 可理解为验证接收到的要计算的类的属性，这里改为带有验证属性的业务实体可能会好理解一点
    def __init__(cls, name, bases, attr_dict):
        super().__init__(name, bases, attr_dict)
        for key, attr in attr_dict.items():
            if isinstance(attr, Validated):
                type_name = type(attr).__name__
                attr.storage_name = '_{}#{}'.format(type_name, key)
                
                
class Entity(metaclass=EntityMeta):
    """带有验证字段的业务实体"""


class LineItem(Entity):
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [33]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)

In [34]:
vars(coconuts)

{'_NonBlank#description': 'Brazilian coconut',
 '_Quantity#weight': 20,
 '_Quantity#price': 17.95}

In [35]:
LineItem.weight.storage_name

'_Quantity#weight'

一、先自定义一个继承自type类的，用于创建带有验证字段的业务实体的元类class ...(type)；
二、自定义一个引用该元类的带有验证字段的业务实体class ...(metaclass=...)；
三、让类继承自该业务实体，由于类是元类的实例，所以在创建类时会调用元类的__init__方法来制定类。

### 可能需要知道类的属性定义的顺序\_ _ _prepare_ _ _

In [36]:
import collections


class EntityMeta(type):    # 元类，用于创建带有验证字段的业务实体
    
    @classmethod
    def __prepare__(cls, name, bases):
        return collections.OrderedDict()    # 类属性存储在里面
    
    def __init__(cls, name, bases, attr_dict):    
        # __prepare__方法返回的映射会传给__new__方法的最后一个参数，然后再传给__init__方法
        super().__init__(name, bases, attr_dict)
        cls._field_names = []    # 在要构建的类中创建一个_field_names属性
        for key, attr in attr_dict.items():     # 按顺序迭代收到的要计算的类的属性
            if isinstance(attr_dict, Validated):
                type_name = type(attr).__name__
                attr.storage_name = '_{}#{}'.format(type_name, key)
                cls._field_names.append(key)    # 类的属性定义的顺序中各个Validated字段被收集在这个类属性中
                
                
class Entity(metaclass=EntityMeta):
    
    @classmethod
    def field_names(cls):
        for name in cls._field_names:
            yield name    # 按照添加字段的顺序产出字段的名称，即按照：description,weight,price的顺序产出属性名称
            
            
class LineItem(Entity):
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [37]:
# for name in LineItem.field_names():    该示例一定要在命令行中进行
#     print(name)